In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv("train_base_inform.csv",encoding="utf-8")
df_test = pd.read_csv("test_base_inform.csv",encoding="utf-8")

In [3]:
df_train2 = pd.read_csv("rightclick_train.csv",encoding="utf-8")
df_test2 = pd.read_csv("rightclick_test.csv",encoding="utf-8")

In [4]:
df_train3 = pd.read_csv("select_train.csv",encoding="utf-8")
df_test3 = pd.read_csv("select_test.csv",encoding="utf-8")
df_train4 = pd.read_csv("traindata_200313.csv",encoding="utf-8")
df_test4 = pd.read_csv("testdata_200313.csv",encoding="utf-8")

/home/realna/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/realna/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/realna/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_train4.drop(["p0_species","p1_species"],1,inplace=True)
df_train3.drop(["p0_species","p1_species"],1,inplace=True)

In [7]:
df_train = pd.merge(df_train, df_train2, on='game_id', how='outer')
df_test = pd.merge(df_test, df_test2, on='game_id', how='outer')

In [8]:
df_test4["game_id"] = df_test["game_id"] #index로 인해 누락

In [9]:
df_train = pd.merge(df_train, df_train3, on='game_id', how='outer')
df_test = pd.merge(df_test, df_test3, on='game_id', how='outer')

In [10]:
df_train = pd.merge(df_train, df_train4, on='game_id', how='outer')
df_test = pd.merge(df_test, df_test4, on='game_id', how='outer')

In [11]:
y_train = pd.read_csv("y_train.csv",encoding="utf-8")
y_train = y_train["winner"]

In [12]:
df_train.drop(["game_id"],1,inplace=True)
df_test.drop(["game_id"],1,inplace=True)

In [13]:
common_columns = list(set(df_train.columns) & set(df_test.columns))

In [14]:
len(common_columns)

1334

In [15]:
df_train = df_train[common_columns]
df_test = df_test[common_columns]

In [20]:
df_train["player0_species"] = df_train["player0_species"].map({'T':1,'P':2,'Z':3}) #종족 변수를 숫자로 변경
df_train["player1_species"] = df_train["player1_species"].map({'T':1,'P':2,'Z':3})

In [21]:
df_test["player0_species"] = df_test["player0_species"].map({'T':1,'P':2,'Z':3}) #종족 변수를 숫자로 변경
df_test["player1_species"] = df_test["player1_species"].map({'T':1,'P':2,'Z':3})

In [22]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [17]:
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb

In [18]:
def lgb_cv(num_leaves,subsample, learning_rate, n_estimators, colsample_bytree, reg_alpha, reg_lambda,
           x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    kf = KFold(n_splits=n_splits)
    models = []
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_index], y_data[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data[valid_index]
        
        model = lgb.LGBMClassifier(
            num_leaves = int(num_leaves), 
            subsample = np.clip(subsample, 0, 1),
            learning_rate = learning_rate, 
            n_estimators = int(n_estimators), 
            colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict_proba(x_valid)[:, 1]
        true = y_valid
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [39]:
func_fixed = partial(lgb_cv, x_data=df_train, y_data=y_train, n_splits=5, output='score') 
lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'num_leaves': (128, 900),
        'subsample': (0, 1),             # subsample,        범위(0~1)
        'learning_rate': (0.0001, 0.1),
        'n_estimators': (200, 2000),
        'colsample_bytree': (0.15, 1),
        'reg_alpha': (1.5, 9.5), 
        'reg_lambda': (10, 120)
    }, 
    random_state=42
)
lgbBO.maximize(init_points=5, n_iter=50)

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.7261   |  0.4684   |  0.09508  |  1.518e+0 |  590.2    |  2.748    |  27.16    |  0.05808  |
|  2        |  0.7276   |  0.8862   |  0.06015  |  1.475e+0 |  143.9    |  9.259    |  101.6    |  0.2123   |
|  3        |  0.733    |  0.3046   |  0.01842  |  747.6    |  533.1    |  4.956    |  42.04    |  0.6119   |
|  4        |  0.7325   |  0.2686   |  0.02929  |  859.5    |  480.1    |  7.781    |  31.96    |  0.5142   |
|  5        |  0.7219   |  0.6536   |  0.00474  |  1.294e+0 |  259.6    |  2.02     |  114.4    |  0.9656   |
|  6        |  0.7229   |  0.5458   |  0.02488  |  207.8    |  148.5    |  4.362    |  23.66    |  0.02236  |
|  7        |  0.7254   |  0.7744   |  0.02507  |  229.4    |  895.6    |  5.277    |  13.84    |  0.4549   |
|  8      

KeyboardInterrupt: 

In [23]:
func_fixed = partial(lgb_cv, x_data=df_train, y_data=y_train, n_splits=5, output='score') 
lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'num_leaves': (128, 900),
        'subsample': (0, 1),             # subsample,        범위(0~1)
        'learning_rate': (0.0001, 0.1),
        'n_estimators': (200, 2000),
        'colsample_bytree': (0.15, 1),
        'reg_alpha': (1.5, 9.5), 
        'reg_lambda': (10, 120)
    }, 
    random_state=42
)
lgbBO.maximize(init_points=5, n_iter=30)

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.7296   |  0.4684   |  0.09508  |  1.518e+0 |  590.2    |  2.748    |  27.16    |  0.05808  |
|  2        |  0.7296   |  0.8862   |  0.06015  |  1.475e+0 |  143.9    |  9.259    |  101.6    |  0.2123   |
|  3        |  0.7359   |  0.3046   |  0.01842  |  747.6    |  533.1    |  4.956    |  42.04    |  0.6119   |
|  4        |  0.7364   |  0.2686   |  0.02929  |  859.5    |  480.1    |  7.781    |  31.96    |  0.5142   |
|  5        |  0.7239   |  0.6536   |  0.00474  |  1.294e+0 |  259.6    |  2.02     |  114.4    |  0.9656   |
|  6        |  0.7252   |  0.5458   |  0.02488  |  207.8    |  148.5    |  4.362    |  23.66    |  0.02236  |
|  7        |  0.7328   |  0.2018   |  0.01486  |  857.3    |  479.9    |  9.106    |  36.6     |  0.721    |
|  8      

KeyboardInterrupt: 

In [24]:
lgbm = lgb.LGBMClassifier(
            colsample_bytree = 0.3009,
            num_leaves = 197, 
            learning_rate = 0.007092, 
            n_estimators = 1980, 
            reg_alpha = 2.181, 
            reg_lambda = 20.94,
            subsample = 0.8421
        )
        
lgbm.fit(df_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.3009,
               importance_type='split', learning_rate=0.007092, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1980, n_jobs=-1, num_leaves=197, objective=None,
               random_state=None, reg_alpha=2.181, reg_lambda=20.94,
               silent=True, subsample=0.8421, subsample_for_bin=200000,
               subsample_freq=0)

In [25]:
y_pred = lgbm.predict_proba(df_test)

In [26]:
lgbm = lgb.LGBMClassifier(
            colsample_bytree = 0.3789,
            num_leaves = 148, 
            learning_rate = 0.01125, 
            n_estimators = 1321, 
            reg_alpha = 2.401, 
            reg_lambda = 13.21,
            subsample = 0.2678
        )
        
lgbm.fit(df_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.3789,
               importance_type='split', learning_rate=0.01125, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1321, n_jobs=-1, num_leaves=148, objective=None,
               random_state=None, reg_alpha=2.401, reg_lambda=13.21,
               silent=True, subsample=0.2678, subsample_for_bin=200000,
               subsample_freq=0)

In [27]:
y_pred2 = lgbm.predict_proba(df_test)

In [28]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=100, random_state=0)
lr.fit(df_train, y_train)

/home/realna/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/realna/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
y_pred3 = lr.predict_proba(df_test)

In [30]:
y_pred[:,1:][:20]

array([[0.49860421],
       [0.62614206],
       [0.38699412],
       [0.21118005],
       [0.59026502],
       [0.66544719],
       [0.69237396],
       [0.64209092],
       [0.33235271],
       [0.54647695],
       [0.5824108 ],
       [0.26276588],
       [0.31025146],
       [0.74332745],
       [0.4288229 ],
       [0.80618628],
       [0.28960062],
       [0.54763952],
       [0.35984306],
       [0.27640501]])

In [31]:
y_pred2[:,1:][:20]

array([[0.49406703],
       [0.63183124],
       [0.4013614 ],
       [0.21266371],
       [0.61626678],
       [0.69668313],
       [0.67434269],
       [0.66358439],
       [0.36720591],
       [0.56051078],
       [0.58668   ],
       [0.25340723],
       [0.31648706],
       [0.76304728],
       [0.40731739],
       [0.8237195 ],
       [0.26612777],
       [0.56791125],
       [0.37890396],
       [0.25698207]])

In [32]:
y_pred3[:,1:][:20]

array([[0.52710932],
       [0.48245273],
       [0.48175313],
       [0.15390668],
       [0.47131885],
       [0.49670984],
       [0.61124066],
       [0.39972549],
       [0.45223174],
       [0.45672028],
       [0.48839961],
       [0.23866171],
       [0.14299644],
       [0.61055016],
       [0.30153567],
       [0.83248353],
       [0.49503066],
       [0.66008406],
       [0.52709969],
       [0.45777922]])

In [33]:
pred = 0.5*y_pred[:,1:] + 0.5*y_pred2[:,1:]

In [34]:
pred = 0.8*pred + 0.2*y_pred3[:,1:]

In [35]:
pred

array([[0.50249036],
       [0.59967987],
       [0.41169283],
       ...,
       [0.77831402],
       [0.5493904 ],
       [0.45274795]])

In [37]:
sample_submission=pd.read_csv('sample_submission.csv',encoding="utf-8") 

In [38]:
sample_submission["winner"] = pred
sample_submission.to_csv("sel_rc_event_2lr_8lgbm_submission35.csv",encoding="utf-8",index=False)

In [39]:
df_train.to_csv("Final_df_train.csv",encoding="utf-8",index=False)
df_test.to_csv("Final_df_test.csv",encoding="utf-8",index=False)